# 암호화폐 가격 예측

Binance Market Data(CandleStick)와 Sequence2Sequence를 이용한 암호화폐 가격 예측.

**TEST CASE**

- Feature에 따른 Test

- Layer에 따른 Test

**무엇을 예측할 것인가**
무엇을 예측할지도 중요한 포인트이다.

- 미래 가격 그래프 예측

- 최종 가격보다 n% 상승 혹은 하락 예측

    - X시간 내에 n%상승 혹은 하락할 것인가
    - X시간 후 Y시간 내에 n% 상승 혹은 하락할 것인가

### Env Check

In [5]:
from tensorflow.python.client import device_lib
import sys
import tensorflow as tf
print('Python version is\t',sys.version.split()[0],sys.version.split()[1],sys.version.split()[2])
print('Tensorflow version is\t',tf.__version__)
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
print("GPU device is\t\t",get_available_gpus())

Python version is	 3.5.6 |Anaconda, Inc.|
Tensorflow version is	 1.12.0
GPU device is		 ['/device:GPU:0']


In [6]:
import pandas as pd
import json
from pprint import pprint as p
import os
import matplotlib.pyplot as plt
import numpy as np
import datetime
import time
import tensorflow as tf
import numpy as np
from binance.client import Client
from datetime import datetime
import csv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def __Api__():
    f = open("/home/scio/zepl/conf/API_KEY", 'r')
    line = f.readline()
    f.close()
    return line
def __Sign__():
    f = open("/home/scio/zepl/conf/SECRET_KEY", 'r')
    line = f.readline()
    f.close()
    return line
    
print('python version is',sys.version.split()[0],sys.version.split()[1],sys.version.split()[2])
print('tensorflow version is',tf.__version__)

python version is 3.5.6 |Anaconda, Inc.|
tensorflow version is 1.12.0


### API를 이용한 Kline(CandleSticks) 호출 및 저장

In [7]:
BTC='BTCUSDT'
client = Client(__Api__(),__Sign__())
klines_orig = client.get_historical_klines(BTC, '4h', "1 Jul, 2014")

for i in range(len(klines_orig)):
    klines_orig[i][0]=int(datetime.fromtimestamp(klines_orig[i][0]/1000).strftime('%Y%m%d%H%M'))
    klines_orig[i][6]=int(datetime.fromtimestamp(klines_orig[i][6]/1000).strftime('%Y%m%d%H%M'))
klines=np.array(klines_orig)


df=pd.DataFrame(klines)
# To JSON
with open("/home/scio/zepl/data/bitcoinKlines/4klines.json", 'w') as f:
    f.write(df.to_json(orient='records', lines=True))
    
# To CSV
df.to_csv("/home/scio/zepl/data/bitcoinKlines/data4klines.csv",sep=',')
print("shape is",klines.shape)

shape is (2702, 12)


현재 dataLoad의 return은 7가지 피쳐(OHLC, QuoteAssetVolume, NumOfTrades)만을 리턴하고있다.
전체 피쳐를 리턴하여 트레이닝 할 경우 loss가 nan이 나온다. normalization의 문제로 추정 中

### Data 로드 및 가공

In [8]:
def dataLoad(filename):
    fp = open(filename, 'r')
    csv_reader = csv.reader(fp)
    btcusdt_data = []
    for row in csv_reader:
        btcusdt_data.append(row)
    del btcusdt_data[0] #인덱스 역할 컬럼 제거
    
    
    for row in btcusdt_data: #Open time
        del row[0] # Open time
        del row[6] # Close time 제거
    dataset = []
    for data in btcusdt_data:
        data_point = []
        is_null = False
        for element in data[1:]:
            if element == 'null':
                is_null = True
            else:
                data_point.append(float(element))
        if is_null:
            continue
        dataset.append(np.array(data_point))
    dataset = np.array(dataset)
    temp= np.array(dataset)

    return dataset [:,0:7]

In [10]:
def timeSlicer(dataset):
    seg_start = 0
    seg_end = 100 #시계열 사이즈 셋팅 
    train_size=1500 #트레인으로 몇개 쓸 지
    num_of_pred=30 #캔들 몇개 예측할 지 
    inputs = []
    labels = []
    while seg_end + num_of_pred <= len(dataset):
        input_segment = dataset[seg_start:seg_end]
        label_segment = dataset[seg_end:seg_end+num_of_pred]
        inputs.append(input_segment)
        labels.append(label_segment)
        seg_start += 1
        seg_end += 1
    inputs = np.array(inputs)
    labels = np.array(labels)

    train_input = inputs[:train_size]
    train_label = labels[:train_size]

    test_input = inputs[train_size:]
    test_label = labels[train_size:]

    return train_input, train_label, test_input, test_label

In [11]:
def read_data():
   dataset = dataLoad("/home/scio/Desktop/bot/data/4klines.csv")
   train_input, train_label, test_input, test_label = timeSlicer(dataset)
   return train_input, train_label, test_input, test_label

### Data Format

Candle Stick Data Format은 아래와 같다.
원본 데이터의 칼럼 중 OpenTime, CloseTime, TakerBuyBaseAssetVolume, TakeBuyQuoteAssetVolum, Ignore은 제외하였다.

In [9]:
dataset = dataLoad("/home/scio/Desktop/bot/data/4klines.csv")
print("Shape of Candle Sticks is",np.shape(dataset))
print("Type of Candle Sticks is",type(dataset))
df = pd.DataFrame(dataset).rename(columns={0:'Open',1:'High',2:'Low',3:'Close',4:'Volume',5:'QuoteAssetVolume',6:'NumOfTrades',7:'TakerBuyBaseAssetVolume',8:'TakeBuyQuoteAssetVolum',9:'Ignore'})
p(df[:5])

Shape of Candle Sticks is (2691, 7)
Type of Candle Sticks is <class 'numpy.ndarray'>
      Open     High      Low    Close      Volume  QuoteAssetVolume  \
0  4261.48  4349.99  4261.32  4349.99   82.088865      3.531943e+05   
1  4333.32  4485.39  4333.32  4427.30   63.619882      2.825012e+05   
2  4436.06  4485.39  4333.42  4352.34  174.562001      7.742388e+05   
3  4352.33  4354.84  4200.74  4325.23  225.109716      9.652911e+05   
4  4307.56  4369.69  4258.56  4285.08  249.769913      1.079545e+06   

   NumOfTrades  
0        334.0  
1        248.0  
2        858.0  
3        986.0  
4       1001.0  


### seq2seq 네트워크 정의

In [12]:
def model(inputs_ph, init_state, seq_len, out_seq_len, sizeOfColumns):
   '''
   arguments:
      inputs_ph: input placeholder
      init_state: 초기 상태 placeholder
      seq_len: input의 time 길이
      out_seq_len: output의 time 길이
   return:
      softmax result
   '''
   with tf.variable_scope('rnn') as scope:
      # encoder
      rc1 = init_state
      for i in range(seq_len):
         input_prev = tf.concat([inputs_ph[i], rc1], 1)
         rc1 = fully_connected_layer(input_prev, NODE_NUM, 'rc1')
         rc1 = tf.nn.tanh(rc1)
         rc1 = tf.nn.tanh(rc1)
         rc1 = tf.nn.tanh(rc1)
         rc1 = tf.nn.tanh(rc1)
         rc1 = tf.nn.tanh(rc1)
         rc1 = tf.nn.tanh(rc1)
         rc1 = tf.nn.tanh(rc1)
         rc1 = tf.nn.tanh(rc1)
         logits = fully_connected_layer(rc1, sizeOfColumns, 'logits')
         if i==0:
            tf.get_variable_scope().reuse_variables()

      # decoder
      logits_list = [tf.reshape(logits, (-1, 1, sizeOfColumns))]
      for i in range(out_seq_len-1):
         input_prev = tf.concat([logits, rc1], 1)
         rc1 = fully_connected_layer(input_prev, NODE_NUM, 'rc1')
         rc1 = tf.nn.tanh(rc1)
         rc1 = tf.nn.tanh(rc1)
         rc1 = tf.nn.tanh(rc1)
         rc1 = tf.nn.tanh(rc1)
         rc1 = tf.nn.tanh(rc1)
         rc1 = tf.nn.tanh(rc1)
         rc1 = tf.nn.tanh(rc1)
         rc1 = tf.nn.tanh(rc1)
         logits = fully_connected_layer(rc1, sizeOfColumns, 'logits')
         logits_list.append(tf.reshape(logits, (-1, 1, sizeOfColumns)))

      result = tf.concat(logits_list, axis=1)

   return result

In [13]:
def fully_connected_layer(input_tensor, out_node_num, name):
   input_dim = input_tensor.shape.as_list()
   W = tf.get_variable(name=name+'_weights', shape=[input_dim[1], out_node_num])
   b = tf.get_variable(name=name+'_biases', shape=[out_node_num])
   result = tf.matmul(input_tensor, W) + b
   return result

Least Square Method을 이용한 Loss 측정

In [14]:
def objective_graph(prediction, labels):
   lsm = tf.reduce_mean(tf.reduce_sum(tf.square(prediction - labels), [1, 2]))
   return lsm

### 셋팅

In [15]:
NODE_NUM = 512
SEQ_LEN = 100 #시계열 사이즈 셋팅
OUT_SEQ_LEN = 30 #캔들 몇개 예측할지
train_input, train_label, test_input, test_label = read_data()

In [16]:
sizeOfColumns = np.shape(train_input)[2] #피쳐 갯수
inputs = tf.placeholder(tf.float32, [None, SEQ_LEN, sizeOfColumns])

inputs_max = tf.reduce_max(inputs, 1, True)
inputs_min = tf.reduce_min(inputs, 1, True)
normed_inputs = (inputs - inputs_min) / (inputs_max - inputs_min)

inputs_t = tf.transpose(normed_inputs, [1, 0, 2])
inputs_r = tf.reshape(inputs_t, [-1, sizeOfColumns])

inputs_s = tf.split(inputs_r, SEQ_LEN, axis=0)

init_state = tf.placeholder(tf.float32, [None, NODE_NUM])
labels = tf.placeholder(tf.float32, [None, OUT_SEQ_LEN, sizeOfColumns])
sizeOfColumns

7

In [17]:
normed_labels = (labels - inputs_min) / (inputs_max - inputs_min)

prediction = model(inputs_s, init_state, SEQ_LEN, OUT_SEQ_LEN, sizeOfColumns)
loss = objective_graph(prediction, normed_labels)

train_vars = tf.trainable_variables()
for var in train_vars:
   print(var.name)

train_op = tf.train.AdamOptimizer(1e-4).minimize(loss)
init_op = tf.global_variables_initializer()

rnn/rc1_weights:0
rnn/rc1_biases:0
rnn/logits_weights:0
rnn/logits_biases:0


실제 트레이닝을 진행한다.
1. 1차로 Loss가 0.15 이하로 나올시 모델과 Loss값을 저장한다.
2. 저장된 Loss값 * 0.9보다 낮은 Loss가 나올 시 다시 모델과 Loss값을 저장한다.

2를 반복한다.

In [ ]:
r=700000

filename=str(r)+'_'+str(SEQ_LEN)+'_'+str(OUT_SEQ_LEN)
now=datetime.now()
time=str(now.year)+str(now.month)+str(now.day)+'_'+str(now.hour)+str(now.minute)+'__'
path="/home/scio/zepl/tensor/bitcoinPred/model/"+time+filename+'/'
os.system('mkdir '+path)
print(path)

lsm_scalar = tf.summary.scalar(filename,loss) #tensorboard loss
summary=tf.summary.merge_all() #tensorboard
saver = tf.train.Saver()

minLoss=0.15

with tf.Session(config=tf.ConfigProto(log_device_placement=False)) as sess:
    with tf.device('/gpu:0'):
        sess.run(init_op)
        writer = tf.summary.FileWriter("/home/scio/zepl/tensor/bitcoinPred/board/",sess.graph) #tensorboard
        
        with tf.name_scope('train'):
            for step in range(r):
            
                shuffle_index = np.random.permutation(len(train_input))
                batch_xs = train_input[shuffle_index[:100]]
                batch_ys = train_label[shuffle_index[:100]]
                feed_dict = {}
                feed_dict[inputs] = batch_xs
                feed_dict[labels] = batch_ys

                init_state_value = np.zeros([len(batch_ys), NODE_NUM])
                feed_dict[init_state] = init_state_value
                sess.run(train_op, feed_dict=feed_dict)
                
                input_value = sess.run(normed_inputs, feed_dict=feed_dict)
                label_value = sess.run(normed_labels, feed_dict=feed_dict)
                pred_value = sess.run(prediction, feed_dict=feed_dict)
                loss_value = sess.run(loss, feed_dict=feed_dict)
                
                # 이전의 Min Loss * 0.9 보다 낮은 Loss가 나올 경우 모델 저장
                if (minLoss*0.9) > loss_value:
                    minLoss=loss_value
                    print("--------------------------------------------------")
                    print(step,'step train loss value: ', loss_value)
                    saver.save(sess, path+str(loss_value))
                show_loss = sess.run(lsm_scalar, feed_dict=feed_dict) #tensorboard loss
                writer.add_summary(show_loss, step) #tensorboard loss
        saver.save(sess, path+"_final")


학습된 모델을 이용하여 실제 테스트 후 이미지로 저장

In [ ]:
os.system("mkdir "+path+"predImg")
for i in range(len(test_label)):
	plt.cla()
	plt.plot(range(-99, 1), input_value[i, :, 1], 'b')
	plt.plot(range(1, 31), label_value[i, :, 1], 'g')
	plt.plot(range(1, 31), pred_value[i, :, 1], 'r--')
	plt.title('test')
	plt.savefig(path+"predImg/test"+str(i)+".png")
		